# 2018210765\_李顺立_L8
----
----

## 中新网爬取中美贸易战新闻
+ 导入模块
+ 使用selenium打开浏览器
+ 定义根据链接查找新闻的函数（使用request）url\_get\_content(url)
+ 定义时间转化函数time\_trans(time\_need)
+ 定义中新网爬取函数china_news(pages=5)
+ 结果预览
----

## 导入模块

In [1]:
import time
from selenium.webdriver.chrome.options import Options
import numpy as np
from selenium import webdriver
import pandas as pd
import re
import os
import requests                        #打开链接
from bs4 import BeautifulSoup          #解码链接
from urllib.parse import urlencode     #输入内容转码
time.sleep(np.random.rand()+0.5)

----
## 使用selenium打开浏览器

In [2]:

##不打开浏览器
options = Options() 
options.add_argument("--headless") 


#打开浏览器并且打开虎扑NBA
driver = webdriver.Chrome()   #options = options
driver.get('http://sou.chinanews.com/search.do')  #打开链接

driver.find_element_by_xpath('//*[@id="q"]').send_keys('中美贸易战') #输入关键词
driver.find_element_by_xpath('/html/body/div[2]/form/table/tbody/tr/td/table/tbody/tr[2]/td[1]/input[2]').click()   #点击搜索

----
## 定义根据链接查找新闻的函数（使用request）url_get_content(url)

In [3]:
def url_get_content(url):
    """
    该函数根据所传入的字符串类型的链接url获取该页的内容
    示例：url = 'https://baidu.com'
    结果：输出为预处理后的结果
    """
    content = ''
    try:
        html = requests.get(url) #, cookies = cookies, headers = headers) #打开链接  #伪装成真实的访问者
        soup = BeautifulSoup(html.content, "html.parser")  #解码链接
        content0 = soup.find('div', {"class": "left_zw"}).get_text() #解码后找到相应内容
        content1 = ','.join(re.findall(r'\w+', content0))  #处理初始数据
        
        #试图深度处理数据
        try:
            remove = re.findall(r'function.*?fix', content1)[0]  
            content = content1.replace(remove, '')
        except:
            content  = content1
            
    except:
        pass
    
    return content

----
## 定义时间转化函数time_trans(time_need)

In [4]:
def time_trans(time_need):
    hour = time_need // 3600 #整除,求小时数
    days = hour // 24 #求天数
    hours = hour % 24 #求小时数
    minutes = (time_need % 3600) // 60 #先取余数，再取整除，求分钟数
    seconds =  (time_need % 3600) % 60    #求秒数
    
    return days,hours,minutes,seconds  #返回时间

In [5]:
url = 'http://www.chinanews.com/business/2019/04-26/8821443.shtml'
aa = url_get_content(url)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


----
## 定义中新网爬取函数china_news(pages=5)

In [6]:
def china_news(pages = 5):


    import time as TM

    time_begin = TM.time()  #初始化时间
    col = 0  #查找的第i个内容
    page = 1    #初始化页数
    column_name = ['title', 'url', 'date_time', 'content']   #所获取的内容
    data = pd.DataFrame(columns=column_name)  #初始化结果
    
    print('-----'*8, '\n准备爬取第1页的内容！！')
    os.system('say "准备爬取第1页的内容!"')
    
    while page <= pages:  #页数还没爬完就继续
        
        key = True  #判断是否爬完了某一页的所有内容
        i = 1   #初始化当前页的新闻个数
        
        while key:  #爬取当前页的内容
            
            try:

                url = '//*[@id="news_list"]/table[%s]'%(i)   #第i个新闻的位置
                title_url = '/tbody/tr[1]/td[2]/ul/li[1]/a'   #第i个新闻的标题位置
                date_time_url = '/tbody/tr[2]/td/ul/li'   #第i个新闻的时间信息，包含了链接和时间


                title = driver.find_element_by_xpath(url+title_url).text  #标题
                href = driver.find_element_by_xpath(url+title_url).get_attribute('href')  #链接
                date_time0 = driver.find_element_by_xpath(url+date_time_url).text #时间+链接
                date_time = re.findall(r'\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d', date_time0)[0]  #只要时间
                content = url_get_content(href)  #获得链接中的内容

                #整理数据到data
                data.loc[col, :] = [title, href, date_time, content]

                i += 1  #更新到下一个新闻
                col += 1  #更新col index
                
                ##每10条数据休息一会并告知我
                if col % 10 == 0:
                    print('已经爬取数据%s条！'%(col))  
                    os.system('say "叮叮"')
                    os.system('say "%s"'%(col))
                    TM.sleep(np.random.rand()+3)   #每爬完一页的内容多休息一会
                
                TM.sleep(np.random.rand()+1)  #每爬完一个新闻暂停一会

            except:
                key = False
                
                ##估算剩余时间
                time_cost = TM.time() - time_begin  #当前所用时间
                print('已经用时%.2f秒!!'%(time_cost))
                time_mean = time_cost/page  #当前每页的平均时间        
                time_need = time_mean*(pages-page)   #剩余页数的时间
                days,hours,minutes,seconds = time_trans(time_need)  #单位为秒的时间数据处理为日期
                print('预计还需用时\"%d天%02.0f:%02.0f:%02.0f\"!'%(days, hours, minutes, seconds))
                print('-----'*4)
                
                page += 1   #更新到下一页
                if page <= pages:
                    print('正在爬取第%s页的内容！！'%(page))
                    os.system('say "正在爬取第%s页的内容"'%(page))
                    ##点击下一页
                    driver.find_element_by_link_text('下一页').click()   #点击下一页
                
                
    print('用时%.2f秒已经爬完%s页的内容共%s条！！'%(TM.time()-time_begin, page-1,col))  #已经爬完数据
    print('******'*8)
    
    driver.quit()  #退出并关闭浏览器
    
    os.system('say "顺立，内容已经爬取完成！请查收！"')  #告知我
    
    return data
        
        
        
        
        
        
        

----
##  结果预览

In [7]:
data = china_news(53)

---------------------------------------- 
准备爬取第1页的内容！！


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


已经爬取数据10条！
已经用时28.43秒!!
预计还需用时"0天00:24:38"!
--------------------
正在爬取第2页的内容！！


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


已经爬取数据20条！
已经用时59.90秒!!
预计还需用时"0天00:25:28"!
--------------------
正在爬取第3页的内容！！


...

已经爬取数据420条！
已经用时1316.04秒!!
预计还需用时"0天00:05:45"!
--------------------
正在爬取第43页的内容！！


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


已经爬取数据430条！
已经用时1345.97秒!!
预计还需用时"0天00:05:13"!
--------------------
正在爬取第44页的内容！！


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


已经爬取数据440条！
已经用时1378.38秒!!
预计还需用时"0天00:04:42"!
--------------------
正在爬取第45页的内容！！
已经爬取数据450条！
已经用时1406.08秒!!
预计还需用时"0天00:04:10"!
--------------------
正在爬取第46页的内容！！
已经爬取数据460条！
已经用时1436.11秒!!
预计还需用时"0天00:03:39"!
--------------------
正在爬取第47页的内容！！
已经爬取数据470条！
已经用时1462.89秒!!
预计还需用时"0天00:03:07"!
--------------------
正在爬取第48页的内容！！
已经爬取数据480条！
已经用时1491.69秒!!
预计还需用时"0天00:02:35"!
--------------------
正在爬取第49页的内容！！
已经爬取数据490条！
已经用时1521.74秒!!
预计还需用时"0天00:02:04"!
--------------------
正在爬取第50页的内容！！
已经爬取数据500条！
已经用时1554.22秒!!
预计还需用时"0天00:01:33"!
--------------------
正在爬取第51页的内容！！
已经爬取数据510条！
已经用时1585.12秒!!
预计还需用时"0天00:01:02"!
--------------------
正在爬取第52页的内容！！
已经爬取数据520条！
已经用时1614.28秒!!
预计还需用时"0天00:00:31"!
--------------------
正在爬取第53页的内容！！
已经用时1625.74秒!!
预计还需用时"0天00:00:00"!
--------------------
用时1625.74秒已经爬完53页的内容共524条！！
************************************************


In [52]:
data.head()

,title,url,date_time,content
0,磋商要平等互利 得适应打打谈谈,http://www.chinanews.com/gn/2019/05-11/8833614...,2019-05-11 07:02:36,"5月11日凌晨,第十一轮中美经贸磋商传出结束的消息,据说,在磋商结束后,刘鹤副总理接受了中国..."
1,国际锐评：解决中美经贸摩擦 关键在照顾彼此关切,http://www.chinanews.com/gn/2019/05-11/8833591...,2019-05-11 03:42:43,"国际锐评,解决中美经贸摩擦,关键在照顾彼此关切,第十一轮中美经贸高级别磋商当地时间星期五,1..."
2,关税大棒损人害己,http://www.chinanews.com/gn/2019/05-10/8833026...,2019-05-10 15:05:31,"国际锐评,关税大棒损人害己,华盛顿时间10日0点01分,北京时间10日12点01分,美方对2..."
3,浙大公布自主招生初审名单,http://www.chinanews.com/sh/2019/05-09/8831841...,2019-05-09 11:32:50,"计划招生不超过197人,一共有1534人通过初审,浙大公布自主招生初审名单学科竞赛省一等奖是..."
4,社评：美国要摆“鸿门宴”，但吓唬不住中国,http://www.chinanews.com/gn/2019/05-09/8831586...,2019-05-09 08:31:29,"美国,联邦公报,于北京时间8日晚上刊登一份公告称,从星期五开始,美国将把2000亿美元中国输..."


In [51]:
data.to_csv('中美贸易战.csv', index = False)